# Consolidación de bases y análisis de los textos

  El código descrito a continuación tiene dos funcionalidades:

1. <p style='text-align: justify;'>La primera es descargar cada uno de los decretos publicados en la página del Miniterio de Salud, con base al Scrapping realizado en el script Scrapping_Decretos_MinSalud.ipynb.</p>
2.  <p style='text-align: justify;'> La segunda parte, transcribe los pdf a texto y valida si los textos tienen menos de 200 palabras para trasnformar dichos pdf en archivos Word y posteriormente reescribir el archivo PDF, esto debido a que hay textos que se publicand y son realizados a traves de escaneos; por tanto, una manera de solucionar eso es la descrita anteriormente. </p>

La finalidad de este script es obtener cada uno de los textos de los decretos y generar un dataframe para exportar como csv para el analisis de texto.

En primera medida, se importan las librerias necesarias para los procesos descritos en las funcionalidades.</p> 

In [1]:
import os
import pandas as pd
import string
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from tika import parser
import PyPDF3
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import win32com.client
import sys
import comtypes.client
plt.rcParams["figure.figsize"] = [18.0, 8.0]

In [2]:
links=pd.read_csv("links_decreto.csv")
links=links.drop(columns=("Unnamed: 0"))

In [3]:
links.head()

,link,nombre,años
0,https://www.minsalud.gov.co/Normatividad_Nuevo...,Decreto_600_de_2020.pdf,2020
1,https://www.minsalud.gov.co/Normatividad_Nuevo...,Decreto_539_de_2020.pdf,2020
2,https://www.minsalud.gov.co/Normatividad_Nuevo...,Decreto_538_de_2020.pdf,2020
3,https://www.minsalud.gov.co/Normatividad_Nuevo...,Decreto_521_de_2020.pdf,2020
4,https://www.minsalud.gov.co/Normatividad_Nuevo...,Decreto_491_de_2020.pdf,2020


In [4]:
links.dtypes

link      object
nombre    object
años       int64
dtype: object

El siguiente código lo que hace es realizar el request de cada ruta de decreto y guarda el PDF de cada una en la ruta donde se encuentra este script. 

In [ ]:
import urllib.request
for i in range(0,len(links)):
    print(links['nombre'][i])
    urllib.request.urlretrieve(links['link'][i], links['nombre'][i])

<p style='text-align: justify;'>  El siguiente código genera una función la cual será usada para extraer texto de cada uno de los textos a través de dos métodos, uno es usando la librería "tika", de alli, "parser" trascribe los textos de pdf a una list; sin embargo, para los archivos que dicha librería no pueda trascribir se usa la librería "PyPDF3" el cual transcribe una por una de las hojas de los PDFs.</p>

In [5]:
def texto(archivopdf):
    try:
        rawText = parser.from_file(archivopdf)
        rawList1 = [rawText['content'].replace("\n", "")]
    except:
        try:
            pdfFileObject = open(archivopdf, 'rb')
            pdfReader = PyPDF3.PdfFileReader(pdfFileObject)
            count = pdfReader.numPages
            rawList1=[]
            
            for i in range(count):
                page = pdfReader.getPage(i)
                texto0=page.extractText()
                rawList1.append(texto0.replace("\n", ""))
        except:
            rawList1=[]
    return rawList1

El siguiente código realiza la trascipción de cada decreto usando la función anterior y guardandolo en una lista. Luego guarda esa lista en una columna de la base de links de decretos publicados por el Ministerio de Salud.

In [ ]:
textos_transcritos=[]
for i in range(len(links)):
    print(i)
    palabras=texto(links['nombre'][i])
    textos_transcritos.append(palabras)

In [ ]:
links['textos']=textos_transcritos

El siguiente código realiza el conteo de cada palabra por cada decreto y lo guarda en una columnda del dataframe.

In [ ]:
links['numero_palabras']=''
for i in range(len(textos_transcritos)):  
    try:
        links['numero_palabras'][i]=len(textos_transcritos[i][0])
    except:
        links['numero_palabras'][i]=0

Para los archivos que transcribio y no supero las 500 palabras, se realiza el proceso de convertir dichos archivos a Word y posteriormente gurda como PDF, reemplazando el PDF inicial.

In [ ]:
links1=links[links['numero_palabras']<=500].reset_index(drop=True)

In [ ]:
links1.head()

Las siguientes funciones realizan las siguientes tareas:
1. A través de la función "pdf_a_word", con el uso la librería "win32com" transforma los pdf en archivos word y los guarda en la misma ruta de este script.
2. A través de la función "eliminacion" elimina el pdf que se guardó como word con la función del punto 1.
3. A través de función "word_pdf", guarda el word creado en el punto 1 como PDF con el mismo nombre del archivo eliminado en el punto 2.

In [ ]:
def pdf_a_word(archivo):
    word = win32com.client.Dispatch("Word.Application")
    word.visible = 0
    pdfs_path = archivo
    
    for i, doc in enumerate(glob.iglob(pdfs_path)):
        filename = doc.split('\\')[-1]
        in_file = os.path.abspath(doc)
        
        print("pdf transformado:\n",in_file)
        wb = word.Documents.Open(in_file)
        out_file = os.path.abspath(filename[0:-4]+ ".docx".format(i))
        
        print("word transformado a pdf:\n",out_file)
        wb.SaveAs2(out_file, FileFormat=16) 
        wb.Close(True)# cerrar la aplicación word
    word.Quit()

In [ ]:
def eliminacion(ruta):
    if os.path.isfile(ruta) or os.path.islink(ruta):
        os.remove(ruta)  # 
    elif os.path.isdir(ruta):
        shutil.rmtree(ruta)  
    else:
        raise ValueError("Archivo {} no esta en la direccion.".format(ruta))

In [ ]:
def word_pdf(archivo):
    wdFormatPDF = 17

    in_file = os.path.abspath(archivo[:-4]+".docx")
    out_file = os.path.abspath(archivo[:-4]+".pdf")

    word = comtypes.client.CreateObject('Word.Application')
    doc = word.Documents.Open(in_file)
    doc.SaveAs(out_file, FileFormat=wdFormatPDF)
    doc.Close()
    word.Quit()

El siguiente código hace la iteración de las funciones descritas anteriormente para cada uno de los registros que no cumplen con una longitud de palabras mayores a 500. Esto dado que algunos PDFs fueoron creador a tarvés de un proceso de escaneo. 

In [ ]:
for i in range(0,len(links1)):
    pdf_a_word(links1['nombre'][i])
    eliminacion(links1['nombre'][i])
    word_pdf(links1['nombre'][i])

El siguiente código tiene como objetivo iterar la función de transcribir de nuevo los textos para volver a validar si efectivamente hubo un aumento en el número de palabras de cada decreto. Luego de esto, guarda las trascripciones en otra columna del dataframe base que se tiene para el análisis. 

In [6]:
textos_transcritos1=[]
for i in range(len(links)):
    print(i)
    palabras=texto(links['nombre'][i])
    textos_transcritos1.append(palabras)
links['textos1']=textos_transcritos1

0


2020-05-15 22:27:21,755 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


El siguiente código realiza el conte de palabras en cada uno de los decretos publicados por el MinSalud, con base al arreglo generado anteriormente.

In [8]:
links['numero_palabras1']=''
for i in range(len(textos_transcritos1)):  
    try:
        links['numero_palabras1'][i]=len(textos_transcritos1[i][0])
    except:
        links['numero_palabras1'][i]=0

C:\Users\LENOVO\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [9]:
links[links['numero_palabras1']<=500]

,link,nombre,años,textos1,numero_palabras1
139,https://www.minsalud.gov.co/Normatividad_Nuevo...,Decreto_3047_de_2013.pdf,2013,[ DECRETO NÚMERO 0482 DE 2013 HOJA No 2 Cont...,383
204,https://www.minsalud.gov.co/Normatividad_Nuevo...,Decreto_1464_de_2012.PDF,2012,[Decreto4465de2011-RL ],27


In [10]:
links.to_excel("Textos_decretos.xlsx")